## Indexação de carga útil: a camada base

In [2]:
from materials.q_drant_client import client
from qdrant_client import QdrantClient, models
# For Colab:
# from google.colab import userdata
# client = QdrantClient(url=userdata.get("QDRANT_URL"), api_key=userdata.get("QDRANT_API_KEY"))

collection_name = "store"
vector_size = 768

if client.collection_exists(collection_name=collection_name):
    client.delete_collection(collection_name=collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=vector_size,
        distance=models.Distance.COSINE,
    ),
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=100,
    ),
)

# Index frequently filtered fields
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

client.create_payload_index(
    collection_name=collection_name,
    field_name="price",
    field_schema=models.PayloadSchemaType.FLOAT,
)

client.create_payload_index(
    collection_name=collection_name,
    field_name="brand",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

collections=[CollectionDescription(name='my_first_collection'), CollectionDescription(name='my_collection'), CollectionDescription(name='day0_first_system'), CollectionDescription(name='dev_vectors'), CollectionDescription(name='production_vectors')]


UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

O indexing_threshold é o gatilho que o Qdrant usa para decidir quando vale a pena gastar processamento (CPU) para construir o índice vetorial (HNSW).

Aqui está a explicação detalhada:

1. O que ele faz?
Ele define a quantidade mínima de vetores que uma coleção deve ter para que o Qdrant comece a construir o índice de busca por aproximação (o grafo HNSW).

No seu exemplo:

python
indexing_threshold=100
Isso diz ao Qdrant: "Não perca tempo construindo o índice HNSW enquanto eu tiver menos de 100 vetores".

2. Por que isso existe?
Construir um índice HNSW é uma operação "cara" para o computador. Se você está inserindo vetores um por um, seria ineficiente o Qdrant tentar atualizar o grafo complexo a cada inserção.

Abaixo do threshold: O Qdrant mantém os vetores em formato "plano" (armazenamento simples) e as buscas são feitas via Brute Force (comparação direta). Isso é extremamente rápido para volumes pequenos.
Acima do threshold: O Qdrant dispara um processo em segundo plano chamado Optimizer, que pega todos esses vetores e começa a "tecer" as conexões do grafo HNSW.
3. A diferença crucial entre os Thresholds
Como você já viu o full_scan_threshold, é comum confundir os dois. Esta tabela mata a dúvida:

Parâmetro	Onde fica	O que controla
indexing_threshold	optimizers_config	O momento da CONSTRUÇÃO do índice. (Quando a CPU deve começar a criar o grafo?)
full_scan_threshold	hnsw_config	A estratégia de BUSCA. (Mesmo que o índice exista, devo usá-lo ou usar Brute Force porque a coleção é pequena?)
4. Por que você usaria o valor 100?
Inserções em massa (Bulk Load): Se você vai inserir 1 milhão de vetores, você não quer que o Qdrant comece a indexar no vetor número 101. Você aumentaria esse número para, por exemplo, 20.000, para que ele receba muita informação antes de começar a calcular o índice, economizando CPU.
Tempo Real: Se você quer que a coleção comece a usar o índice o mais rápido possível após poucas inserções, você usa um valor baixo como 100.
Resumo: O indexing_threshold é o parâmetro de paciência do Qdrant. Ele diz ao banco de dados: "Espere eu ter pelo menos X pontos antes de começar a trabalhar duro na organização do índice HNSW".

In [3]:
# Upload data
import random

points = []
for i in range(1000):
    points.append(
        models.PointStruct(
            id=i,
            vector=[random.random() for _ in range(vector_size)],
            payload={
                "category": random.choice(["laptop", "phone", "tablet"]),
                "price": random.randint(0, 1000),
                "brand": random.choice(
                    ["Apple", "Dell", "HP", "Lenovo", "Asus", "Acer", "Samsung"]
                ),
            },
        )
    )
client.upload_points(
    collection_name=collection_name,
    points=points,
)

## Configurando a pesquisa com filtros

In [4]:
# Create filter combining multiple conditions
filter_conditions = models.Filter(
    must=[
        models.FieldCondition(key="category", match=models.MatchValue(value="laptop")),
        models.FieldCondition(key="price", range=models.Range(lte=1000)),
        models.FieldCondition(key="brand", match=models.MatchAny(any=["Apple", "Dell", "HP"])),
    ]
)

query_vector = [random.random() for _ in range(vector_size)]

# Execute filtered search
results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    query_filter=filter_conditions,
    limit=10,
    search_params=models.SearchParams(hnsw_ef=128),
)

In [5]:
results

QueryResponse(points=[ScoredPoint(id=208, version=10, score=0.76948094, payload={'category': 'laptop', 'price': 141, 'brand': 'Dell'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=284, version=11, score=0.76461315, payload={'category': 'laptop', 'price': 838, 'brand': 'Dell'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=435, version=13, score=0.7638649, payload={'category': 'laptop', 'price': 845, 'brand': 'Apple'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=373, version=12, score=0.76338494, payload={'category': 'laptop', 'price': 432, 'brand': 'Apple'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=952, version=21, score=0.7630521, payload={'category': 'laptop', 'price': 681, 'brand': 'Dell'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=825, version=19, score=0.7624195, payload={'category': 'laptop', 'price': 893, 'brand': 'Dell'}, vector=None, shard_key=None, order_value=None), ScoredPoint(i